# Regression Week 2: Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using graphlab create. Now we will use graphlab along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a graphlab SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [89]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1449449092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1449449092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to zg33@columbia.edu and will expire on October 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-12133 - Server binary: /Users/zephaniahgrunschlag/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1449449093.log
[INFO] GraphLab Server Version: 1.7.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the other Week 2 notebook. For this notebook, however, we will work with the existing features.

# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in graphlab functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use graphlab's built in 
```
.to_dataframe()
```
which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's 
```
.as_matrix() 
```
to convert the dataframe into a numpy matrix.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

**Please note you will need GraphLab Create version at least 1.7.1 in order for .to_numpy() to work!**

In [8]:
blah = ['date', 'price']
sales[blah]

date,price
2014-10-13 00:00:00+00:00,221900.0
2014-12-09 00:00:00+00:00,538000.0
2015-02-25 00:00:00+00:00,180000.0
2014-12-09 00:00:00+00:00,604000.0
2015-02-18 00:00:00+00:00,510000.0
2014-05-12 00:00:00+00:00,1225000.0
2014-06-27 00:00:00+00:00,257500.0
2015-01-15 00:00:00+00:00,291850.0
2015-04-15 00:00:00+00:00,229500.0
2015-03-12 00:00:00+00:00,323000.0


In [9]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list 
    # into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]

    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [10]:
# the [] around 'sqft_living' makes it a list:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 

# this accesses the first row of the data the ':' indicates 'all columns':
print example_features[0,:] 

print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrays we can use 
```
np.dot()
```
to compute this:

In [11]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [12]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns 
    # and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

If you want to test your code run the following cell:

In [13]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(  w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[1]\*[feature_k] - output  )^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(   w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[1]\*[feature_k] - output  ) \* [feature_i]

The term inside the parenthesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [14]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length 
    #(number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2 * np.dot(errors, feature)
    return(derivative)

To test your feature derivartive run the following:

In [15]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [16]:
from math import sqrt 
# recall that the magnitude/length of a vector 
# [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [85]:
def regression_gradient_descent(feature_matrix, output, initial_weights, 
                                step_size, tolerance, max_steps=10, verbose=True):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    step_num = 1
    while not converged and step_num <= max_steps:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        
        # compute the errors as predictions - output
        errors = predictions - output

        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])

            # add the squared value of the derivative to the gradient magnitude 
            # (for assessing convergence)
            gradient_sum_squares += derivative**2

            # subtract the step size times the derivative from the current weight
            weights[i] -= step_size * derivative
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        step_num += 1
        if gradient_magnitude < tolerance:
            converged = True
        if verbose or converged or step_num > max_steps:
            print("gradient_magnitude for step no. %d is %s" % (step_num, gradient_magnitude))
    return(weights)

In [120]:
def regression_gradient_descent_PhilipCho(feature_matrix, output, initial_weights, 
                                step_size, tolerance, max_steps=10, verbose=True):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    step_num = 1
    RSS_prev = float("inf")
    while not converged and step_num <= max_steps:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        
        # compute the errors as predictions - output
        errors = predictions - output

        gradient_sum_squares = 0 # initialize the gradient sum of squares
        delta = np.zeros(len(weights))
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])

            # add the squared value of the derivative to the gradient magnitude 
            # (for assessing convergence)
            gradient_sum_squares += derivative**2

            # subtract the step size times the derivative from the current weight
            delta[i] = -step_size * derivative
            weights[i] += delta[i]
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        RSS = np.dot(errors, errors)
        if verbose:
            print("RSS is %s" % RSS)
        if RSS/RSS_prev >= 0.999999999: #- tolerance:
            """ This is bad, so undo the last step and reset the step size
                to half the size
            """
            weights -= delta
            step_size /= 2
            if verbose:
                print("resetting step_size to %s" % step_size)
        else:
            RSS_prev = RSS
        if gradient_magnitude < tolerance:
            converged = True
        if verbose or converged or step_num == max_steps:
            print("gradient_magnitude for step no. %d is %s" % (step_num, gradient_magnitude))
        step_num += 1
    return(weights)

#CLOSED FORM SOLUTION
I'm having problems with they're hacked up algorithm and arbitrary choices for the initial weights. So let's also calculate the closed form solution

In [181]:
def closed_from_solution(feature_matrix, output):
    """
    w = (H^T H)^-1 H^T y
    """
    H = feature_matrix
    y = output
    cof = np.dot(H.transpose(), H)
    w = np.linalg.inv(cof).dot(H.transpose()).dot(y)
    return w

In [180]:
m = np.array([[1.0, 2.0],[3.0, 4.0]])
m.transpose()
np.linalg.inv(m)
np.dot(m.transpose(), m)
m.dot(m)
m.dot(np.array([1,0]))

array([ 1.,  3.])

In [96]:
float("inf") < 5

False

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [18]:
train_data, test_data = sales.random_split(.8,seed=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimat the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [184]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])# np.zeros(2) # 
step_size = 7e-12
tolerance = 2.5e7

In [126]:
max_sqft_living = train_data['sqft_living'].max()
max_sqft_living
max_price = train_data['price'].max()
max_price
train_data['sqft_living_scaled'] = train_data['sqft_living'] / max_sqft_living
train_data['price_scaled'] = train_data['price'] / max_price

In [169]:
# let's test out the gradient descent
simple_features_BC = ['sqft_living_scaled']
my_output = 'price_scaled'
(simple_feature_matrix_BC, output_BC) = get_numpy_data(train_data, simple_features_BC, my_output)
initial_weights_BC = np.array([-0.006118, 0.49580186]) #np.zeros(2) # np.array([-47000., 1.])
step_size_BC = 1e-6
tolerance_BC = 0.00001

Next run your gradient descent with the above parameters.

In [99]:
weights_simple = regression_gradient_descent(simple_feature_matrix, output, 
                                            initial_weights, step_size, tolerance,
                                            max_steps=1000, verbose=False)

gradient_magnitude for step no. 1001 is 268805109.282


In [183]:
weights_simple_closed = closed_from_solution(simple_feature_matrix, output)
print(weights_simple_closed)

[-47116.07657494    281.95883857]


In [170]:
weights_simple_BC = regression_gradient_descent(simple_feature_matrix_BC, output_BC, 
                                            initial_weights_BC, step_size_BC, tolerance_BC,
                                            max_steps=100000, verbose=False)

gradient_magnitude for step no. 29267 is 9.99873271421e-06


In [172]:
print(weights_simple_BC)
print(max_price, max_sqft_living)
print(max_price*weights_simple_BC[0], max_price*weights_simple_BC[1]/max_sqft_living)

[-0.00611896  0.49580808]
(7700000.0, 13540.0)
(-47116.001960727524, 281.95880285832976)


In [182]:
weights_simple_BC_closed = closed_from_solution(simple_feature_matrix_BC, output_BC)
print(weights_simple_BC_closed)

[-0.00611897  0.49580814]


In [185]:
weights_simple_PC = regression_gradient_descent_PhilipCho(simple_feature_matrix, output, 
                                            initial_weights, step_size, tolerance,
                                            max_steps=1000, verbose=True)

RSS is 8.30218845259e+15
gradient_magnitude for step no. 1 is 5.05515307744e+13
RSS is 1.68073263529e+15
gradient_magnitude for step no. 2 is 1.31274510235e+13
RSS is 1.23420774087e+15
gradient_magnitude for step no. 3 is 3.40899608253e+12
RSS is 1.20409586112e+15
gradient_magnitude for step no. 4 is 8.852635801e+11
RSS is 1.20206523458e+15
gradient_magnitude for step no. 5 is 2.2988926572e+11
RSS is 1.20192829713e+15
gradient_magnitude for step no. 6 is 59698688259.7
RSS is 1.2019190626e+15
gradient_magnitude for step no. 7 is 15502826422.1
RSS is 1.20191843987e+15
gradient_magnitude for step no. 8 is 4025844401.5
RSS is 1.20191839787e+15
gradient_magnitude for step no. 9 is 1045449748.16
RSS is 1.20191839504e+15
gradient_magnitude for step no. 10 is 271487891.86
RSS is 1.20191839485e+15
resetting step_size to 3.5e-12
gradient_magnitude for step no. 11 is 70504114.6976
RSS is 1.20191839485e+15
resetting step_size to 1.75e-12
gradient_magnitude for step no. 12 is 70504114.6976
RSS is 1

In [186]:
print(weights_simple)
simple_feature_matrix
output
print("I don't like this at all... but I'll go with it for now")
Q2_2_A1_simple_weights_sqft_living = weights_simple[1]
Q2_2_A1_simple_weights_sqft_living

[  -1.7759716   263.02501707]
I don't like this at all... but I'll go with it for now


263.02501706578789

How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [51]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [52]:
predict_simple_features = predict_output(test_simple_feature_matrix, weights_simple)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [60]:
print("predicted: %s" % predict_simple_features)
print("actual: %s" % test_output)
Q2_2_A2_predict_first_house_model_1 = predict_simple_features[0]
Q2_2_A2_predict_first_house_model_1

predicted: [ 356134.44317093  784640.86422788  435069.83652353 ...,  663418.65300782
  604217.10799338  240550.4743332 ]
actual: [ 310000.  650000.  233000. ...,  610685.  400000.  402101.]


356134.44317092974

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [65]:
residuals_simple_features = test_output - predict_simple_features
RSS_model1_simple = np.dot(residuals_simple_features, residuals_simple_features)
print(RSS_model1_simple)


2.75400047593e+14


# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [55]:
# sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
model_features = ['sqft_living', 'sqft_living15'] 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [56]:
weights_better_model = regression_gradient_descent(feature_matrix, output, 
                                            initial_weights, step_size, tolerance,
                                            max_steps = 300)

gradient_magnitude for step no. 1 is 7.30720205489e+13
gradient_magnitude for step no. 2 is 2.26732209651e+13
gradient_magnitude for step no. 3 is 7.0607945821e+12
gradient_magnitude for step no. 4 is 2.27568239427e+12
gradient_magnitude for step no. 5 is 9.28984105638e+11
gradient_magnitude for step no. 6 is 6.56307425178e+11
gradient_magnitude for step no. 7 is 6.10615351821e+11
gradient_magnitude for step no. 8 is 5.93078765307e+11
gradient_magnitude for step no. 9 is 5.78705920128e+11
gradient_magnitude for step no. 10 is 5.64945676163e+11
gradient_magnitude for step no. 11 is 5.51538681425e+11
gradient_magnitude for step no. 12 is 5.38452422879e+11
gradient_magnitude for step no. 13 is 5.25676912708e+11
gradient_magnitude for step no. 14 is 5.1320454369e+11
gradient_magnitude for step no. 15 is 5.01028100319e+11
gradient_magnitude for step no. 16 is 4.89140559101e+11
gradient_magnitude for step no. 17 is 4.77535065233e+11
gradient_magnitude for step no. 18 is 4.66204926754e+11
gra

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [59]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
test_predictions = predict_output(test_feature_matrix, weights_better_model)
test_predictions

array([ 366651.41203656,  762662.39786164,  386312.09499712, ...,
        682087.39928241,  585579.27865729,  216559.20396617])

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [61]:
Q2_2_A3_predict_1st_house_model2 = test_predictions[0]
Q2_2_A3_predict_1st_house_model2

366651.41203655908

What is the actual price for the 1st house in the test data set?

In [64]:
actual_price_of_1st_house = test_output[0]
print("Actual price: $%s" % actual_price_of_1st_house)
Q2_2_A4_price_diffs = {"1st": actual_price_of_1st_house - Q2_2_A2_predict_first_house_model_1,
                      "2nd": actual_price_of_1st_house - Q2_2_A3_predict_1st_house_model2 }
Q2_2_A4_price_diffs

Actual price: $310000.0


{'1st': -46134.443170929735, '2nd': -56651.412036559079}

**Quiz Question: Which estimate was closer to the true price for the 1st house on the Test data set, model 1 or model 2?**

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [70]:
model2_residuals = test_output - test_predictions
RSS_model2 = np.dot(model2_residuals, model2_residuals)
Q2_2_A5_RSS_comparison = {"model 1": RSS_model1_simple, "model 2": RSS_model2}
print("model 1 RSS %s \nmodel 2 RSS %s" % (RSS_model1_simple, RSS_model2))

model 1 RSS 2.75400047593e+14 
model 2 RSS 2.70263446465e+14


**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **

In [71]:
Q2_2_A1_simple_weights_sqft_living

281.91211911641625

In [78]:
Q2_2_A2_predict_first_house_model_1

356134.44317092974

In [79]:
Q2_2_A3_predict_1st_house_model2

366651.41203655908

# got 5 / 5